### Multi Turn Program Synthesis

- Use Dropdown to select the model and device, click the button to load it
    - This can take some time depending on the model size

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
from IPython.display import display, clear_output

from tools.hf_model_utils import load_model, truncate, generate

In [ ]:
tokenizer, model, model_name, device, precision = None, None, None, None, None

model_chooser = widgets.Dropdown(
    options=["Salesforce/codegen-350M-mono", "Salesforce/codegen-2B-mono", "Salesforce/codegen-6B-mono", "Salesforce/codegen-16B-mono"],
    value="Salesforce/codegen-350M-mono",
    description='Model:',
    disabled=False
)

precision_chooser = widgets.Dropdown(
    options=["float32", "float16"],
    value="float16",
    description='Precision:',
    disabled=False
)

device_chooser = widgets.Dropdown(
    options=["cpu", "cuda:0"],
    value="cuda:0",
    description='Device:',
    disabled=False
)

model_button = widgets.Button(
    description="Load Model",
    disabled=False,
    tooltip="Loads the model, selected in the dropdown"
)

chooser_output = widgets.Output()

def on_model_button_clicked(b):
    global tokenizer, model, device, precision, model_name

    model_name = model_chooser.value
    device = device_chooser.value
    precision = precision_chooser.value
    
    with chooser_output:
        clear_output()
        print(f"Loading model {model_name}")
        tokenizer, model = load_model(model_name, device, precision)
        print("Loading finished")


model_button.on_click(on_model_button_clicked)

        
chooser_layout = HBox([model_chooser, device_chooser, precision_chooser, model_button])
chooser_layout_main = VBox([chooser_layout, chooser_output])

display(chooser_layout_main)


In [ ]:
text_area = widgets.Textarea(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me'
)

output = widgets.Output()

layout = HBox([text_area, button])
layout_main = VBox([layout, output])

# multi_turn = True

_wrap = lambda prompt: f"# {prompt}\n"

# histories = ["# Import libraries.\n\nimport numpy as np\n\n"]
histories = ["# Import libraries.\n\nimport numpy as np\n\n"]
prompt_count = 0

def on_button_clicked(b):
    with output:
        print("Generating...")
    
    global histories, prompt_count
    current_input = text_area.value
    
    histories = [h + _wrap(current_input) for h in histories]
    completions = generate(histories, tokenizer, model, device)
    histories = [h + f"{truncate(c, model_name)}\n\n" for h, c in zip(histories, completions)]
    
    with output:      
        print("-" * 10)
        print(prompt_count)
        print("-" * 10)
        print(histories[0])
        print("-" * 10)
        
    prompt_count += 1


button.on_click(on_button_clicked)

display(layout_main)